In [2]:
Today = '31.12.2023'

Data_file = 'OPRECConso full 2023 12 31' + '.xlsx'
Sheet_in_data_file = 'Export Worksheet'

from datetime import date

Output_file = "_".join([str(date.today()),'Deals.xlsx'])
Sheet_in_output_file = 'Deals'

In [3]:
import pandas as pd
import numpy as np

data_export = pd.read_excel(Data_file, sheet_name = Sheet_in_data_file)

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping

In [67]:

data_work.shape[0], data_work[~data_work.customer.isna()].shape[0]

data_work[['customer', 'Cpty_code', 'Cpty_business_segment', 'business_segment', 'businessUnit']]


,customer,Cpty_code,Cpty_business_segment,business_segment,businessUnit
0,COUNTRYSIDE FEED LLC,COUNTRYSIDE FEED LLC,External,NAMSALES,USBFC
1,COUNTRYSIDE FEED LLC,COUNTRYSIDE FEED LLC,External,NAMSALES,USBFC
2,COUNTRYSIDE FEED LLC,COUNTRYSIDE FEED LLC,External,NAMSALES,USBFC
3,COUNTRYSIDE FEED LLC,COUNTRYSIDE FEED LLC,External,NAMSALES,USBFC
4,RUEMO,RUEMO,RUFERT,RUFERT,RUMNM
...,...,...,...,...,...
15686,IP GLAVA KFH CHIKOV ANDREY GEORGIEVICH,IP GLAVA KFH CHIKOV ANDREY GEORGIEVICH,External,RUFERT,RUEMO
15687,CHTRD,CHTRD,TRADING,EURSALES,FREAF
15688,D.2.N SAS,D.2.N SAS,External,EURSALES,FREAF
15689,CHTRD,CHTRD,TRADING,EURPROD,BEANF


In [66]:
data_work = data_export

data_work = data_work.query("rowType == 'free'").reset_index(drop=True)

customer_isntna_index_list = data_work[~data_work.customer.isna()].index.to_list()
data_work = data_work.iloc[customer_isntna_index_list, :].reset_index(drop=True)

# Добавление столбцов
data_work['business_segment'] = merge_SalesUnits(data_work, col='businessUnit', merge_col='ocpSegment').fillna('External')

data_work['Cpty_code'] = merge_Mapping(data_work, 'customer')
data_work.loc[data_work['Cpty_code'].isna(), 'Cpty_code'] = data_work.customer
data_work['Cpty_business_segment'] = merge_SalesUnits(data_work, col='Cpty_code', merge_col='ocpSegment').fillna('External')

data_work['sum_position_money'] = data_work.dealPrice * data_work.unshippedDealAmount *-1

In [70]:
data_work[['businessUnit', 'customer']] = data_work[['businessUnit', 'customer']].apply(lambda x: x.str.upper())
data_for_print = data_work[['productGroupNine', 'businessUnit', 'business_segment', 'Cpty_business_segment', 'dealCurrency', 'unshippedDealAmount', 'sum_position_money']]

data_for_print.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)

In [72]:
names = data_work.loc[data_work.Cpty_business_segment == 'External' , ['customer', 'Cpty_code', 'Cpty_business_segment']].drop_duplicates()

with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a') as writer:
    names.to_excel(writer, sheet_name='Ex_names', index=False)
